In [1]:
from datasets import load_dataset


In [2]:
dataset = load_dataset('cardiffnlp/tweet_topic_single')

Found cached dataset tweet_topic_single (C:/Users/schmi/.cache/huggingface/datasets/cardiffnlp___tweet_topic_single/tweet_topic_single/1.0.4/832eaa087889d9f4bc549869b44e0acb85a78364dfb3d2bc0bdf23a7224cf2ce)


  0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
import torch
from transformers import AutoModel, AutoTokenizer , AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# Load pre-trained TweetRoBERTa model and tokenizer
model_name = "vinai/bertweet-base"
model = AutoModel.from_pretrained("vinai/bertweet-large")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

train_texts = dataset["train_all"]["text"][0]

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
print(train_encodings)

Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [0, 133, 25522, 1039, 36206, 226, 9936, 33481, 1039, 24303, 1451, 5, 25522, 1039, 347, 29895, 14266, 229, 47909, 1039, 24303, 204, 12, 288, 11, 2436, 112, 9, 5, 2027, 2925, 6688, 4, 9771, 7020, 478, 10, 132, 12, 2962, 11683, 4, 26218, 3533, 3005, 4277, 35, 195, 6442, 6, 155, 289, 6, 321, 248, 6, 321, 8112, 6, 158, 229, 849, 19454, 574, 5785, 10816, 849, 19454, 574, 31673, 4929, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [4]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer , AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Load pre-trained TweetRoBERTa model and tokenizer
model_name = "vinai/bertweet-large"
model = RobertaForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=6)
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large") 

# Load and preprocess your existing train, test, and validation datasets
train_texts = dataset["train_all"]["text"]  # List of training tweet texts
train_labels = dataset["train_all"]["label"]  # List of training labels (topic classes)
test_texts = dataset["test_2021"]["text"] # List of testing tweet texts
test_labels = dataset["test_2021"]["label"]  # List of testing labels (topic classes)


# Tokenize and encode the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create PyTorch datasets
train_dataset = TensorDataset(torch.tensor(train_encodings["input_ids"]),
                              torch.tensor(train_encodings["attention_mask"]),
                              torch.tensor(train_labels))
test_dataset = TensorDataset(torch.tensor(test_encodings["input_ids"]),
                             torch.tensor(test_encodings["attention_mask"]),
                             torch.tensor(test_labels))

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
model.to(device)
model.train()

for epoch in range(8):  # Number of training epochs
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted_labels = torch.max(logits, 1)
        epoch_accuracy += accuracy_score(labels.cpu().numpy(), predicted_labels.cpu().numpy())

    epoch_loss /= len(train_loader)
    epoch_accuracy /= len(train_loader)

    print(f"Epoch {epoch + 1} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}")

    # Evaluation on the test set
    model.eval()
    test_accuracy = 0.0
    test_f1_score_macro = 0.0
    test_f1_score_micro = 0.0

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, 1)
            test_accuracy += accuracy_score(labels.cpu().numpy(), predicted_labels.cpu().numpy())
            test_f1_score_macro += f1_score(labels.cpu().numpy(), predicted_labels.cpu().numpy(), average= 'macro')
            test_f1_score_micro += f1_score(labels.cpu().numpy(), predicted_labels.cpu().numpy(), average= 'micro')
            

    test_accuracy /= len(test_loader)
    print(f"test_2021 Accuracy: {test_accuracy:.4f}")

    test_f1_score_macro /= len(test_loader)
    print(f"test_2021 F1 Score (macro): {test_accuracy:.4f}")

    test_f1_score_micro /= len(test_loader)
    print(f"test_2021 F1 Score (micro): {test_accuracy:.4f}")


Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'c

Epoch 1 - Loss: 0.6832 - Accuracy: 0.7626
test_2021 Accuracy: 0.8753
test_2021 F1 Score (macro): 0.8753
test_2021 F1 Score (micro): 0.8753
Epoch 2 - Loss: 0.2724 - Accuracy: 0.9148
test_2021 Accuracy: 0.8923
test_2021 F1 Score (macro): 0.8923
test_2021 F1 Score (micro): 0.8923
Epoch 3 - Loss: 0.1313 - Accuracy: 0.9608
test_2021 Accuracy: 0.8958
test_2021 F1 Score (macro): 0.8958
test_2021 F1 Score (micro): 0.8958
Epoch 4 - Loss: 0.0476 - Accuracy: 0.9861
test_2021 Accuracy: 0.8770
test_2021 F1 Score (macro): 0.8770
test_2021 F1 Score (micro): 0.8770
Epoch 5 - Loss: 0.0184 - Accuracy: 0.9968
test_2021 Accuracy: 0.8946
test_2021 F1 Score (macro): 0.8946
test_2021 F1 Score (micro): 0.8946
Epoch 6 - Loss: 0.0102 - Accuracy: 0.9970
test_2021 Accuracy: 0.8946
test_2021 F1 Score (macro): 0.8946
test_2021 F1 Score (micro): 0.8946
Epoch 7 - Loss: 0.0052 - Accuracy: 0.9989
test_2021 Accuracy: 0.8958
test_2021 F1 Score (macro): 0.8958
test_2021 F1 Score (micro): 0.8958
Epoch 8 - Loss: 0.0014 - Ac

Epoch 1 - Loss: 0.7067 - Accuracy: 0.7553
test_2020 Accuracy: 0.8993
test_2021 Accuracy: 0.8786
Epoch 2 - Loss: 0.2709 - Accuracy: 0.9083
test_2020 Accuracy: 0.8984
test_2021 Accuracy: 0.8875
Epoch 3 - Loss: 0.1194 - Accuracy: 0.9629
test_2020 Accuracy: 0.8837
test_2021 Accuracy: 0.8809
Epoch 4 - Loss: 0.0500 - Accuracy: 0.9859
test_2020 Accuracy: 0.8967
test_2021 Accuracy: 0.8927
Epoch 5 - Loss: 0.0190 - Accuracy: 0.9959
test_2020 Accuracy: 0.8941
test_2021 Accuracy: 0.8927
Epoch 6 - Loss: 0.0065 - Accuracy: 0.9993
test_2020 Accuracy: 0.9019
test_2021 Accuracy: 0.8968
Epoch 7 - Loss: 0.0026 - Accuracy: 0.9995
test_2020 Accuracy: 0.8932
test_2021 Accuracy: 0.8980
Epoch 8 - Loss: 0.0046 - Accuracy: 0.9995
test_2020 Accuracy: 0.8958
test_2021 Accuracy: 0.8998
Epoch 9 - Loss: 0.0144 - Accuracy: 0.9959
test_2020 Accuracy: 0.8845
test_2021 Accuracy: 0.8900
Epoch 10 - Loss: 0.0376 - Accuracy: 0.9906
test_2020 Accuracy: 0.9097
test_2021 Accuracy: 0.8905